In [10]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [11]:
import pandas as pd

In [12]:
file_path = '/content/drive/My Drive/data-challenge/Y_test_random_sEE2QeA.csv'
data = pd.read_csv(file_path, sep=',')

In [13]:
data

,ID,HOME_WINS,DRAW,AWAY_WINS
0,12303,0,1,0
1,12304,0,1,0
2,12305,0,1,0
3,12306,0,1,0
4,12307,1,0,0
...,...,...,...,...
25363,37666,0,1,0
25364,37667,0,1,0
25365,37668,0,0,1
25366,37669,1,0,0
